In [206]:
import time
import re

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="My-app")
location = geolocator.geocode("VIALE DELLA VITTORIA N. 217 92100")

tmp=re.search('(\w*), Sicilia', location.raw['display_name'],2)
tmp[1]

'Agrigento'